# Схемы разделения секрета

In [1]:
IS_DEBUG = 1

In [2]:
def trace(*args, **kwargs):
    """
    Отладочная трассировка
    """
    
    global IS_DEBUG
    if IS_DEBUG:
        print('[TRACE]', end=' ')
        print(*args, **kwargs)

---

In [3]:
# Генерация простых чисел и КГПСЧ
from Crypto.Util.number import getStrongPrime, getPrime
from Crypto.Random import random

# Утилитки
from functools import reduce

In [4]:
def prod(lst):
    """
    Произведение всех элементов списка.
    """
    
    return reduce((lambda x, y: x * y), lst)

----

In [5]:
class GenericParticipant(object):
    def __init__(self, x, y, t):
        """
        Инициализация участника разделения секрета.
        Сохраняются х, у и t.
        """
        
        self._x = x
        self._y = y
        self._t = t
        
        trace('[GenericParticipant]', f'{x = }, {y = }')
        self._verify()
        
        
    def __eq__(self, other):
        """
        Сравнение на равенство. Сравниваю по х - достаточно.
        """
        
        return self.x == other.x
        
        
    @property
    def x(self):
        """
        Получение х.
        """
        
        return self._x
    
    
    @property
    def y(self):
        """
        Получение у.
        """
        
        return self._y
        
        
    @property
    def equation(self):
        """
        Получение соответствующего участнику уравнения.
        """
        
        return [1] + [self._x ^ i for i in range(1, self._t)], self._y
    
    
    def _verify(self):
        pass

In [6]:
class GenericDealer(object):
    def __init__(self, t, n, field):
        """
        Инициализация дилера для разделения секрета.
        Генерация простого числа, поля и ключа.
        """
        
        self._field = field
        self._t = t
        self._n = n
        self._key = self._field.random_element()
        
        self._coefs = []
        self._values = []
        
        trace('[GenericDealer]', f't = {self._t}, n = {self._n}')
        trace('[GenericDealer]', self._field)
        trace('[GenericDealer]', f'key = {self._key}')
    
    
    @property
    def field(self):
        """
        Получение поля.
        """
        
        return self._field
    
    
    @property
    def key(self):
        """
        Получение ключа для проверки
        """
        
        return self._key
    
    
    def _prepare_coefficients(self):
        self._coefs  = [self._key] + [self._field.random_element() for _ in range(self._t - 1)]
        
        self._values = [sum([c * (x ^ i) for i, c in enumerate(self._coefs)]) 
                        for x in range(1, self._n + 1)]

----

In [7]:
def restore_secret_by_equation_system(field, participants: list[GenericParticipant]):
    """
    Восстановление секрета путем решения системы уравнений.
    """
    
    equations = [p.equation for p in participants]
    M = Matrix(field, [eq[0] for eq in equations])
    v = vector(field, (eq[1] for eq in equations))
    
    return M.solve_right(v)[0]  # Нас интересует только первая переменная,
                                # так как именно она является ключом

In [8]:
def restore_secret_by_lagrange_interpolation(participants: list[GenericParticipant]):
    """
    Восстановление секрета путем интерполяции Лагранжа.
    """
    
    return sum([p1.y * prod([p2.x * (p2.x - p1.x).inverse_of_unit() 
                             for p2 in participants if p1 != p2])
                for p1 in participants])

----

## Схема Шамира
![Shamir scheme](./images/Shamir_1_5.png)
![Shamir scheme](./images/Shamir_2_5.png)

In [9]:
class ShamirParticipant(GenericParticipant):
    def __init__(self, x, y, t):
        """
        Тут в целом все повторяет общий случай.
        """
        
        trace('[ShamirParticipant]', f'Initializing superclass...')
        super(ShamirParticipant, self).__init__(x, y, t)

In [10]:
class ShamirDealer(GenericDealer):
    def __init__(self, t, n):
        """
        Тут в целом все повторяет общий случай.
        """
        
        trace('[ShamirDealer]', f'Initializing superclass...')
        super(ShamirDealer, self).__init__(t, n, GF(getStrongPrime(512)))
        
        
    def split_key(self):
        """
        Разделение секрета между участниками.
        """
        
        self._prepare_coefficients()
        return [ShamirParticipant(self._field.gen() + i, self._values[i], self._t) 
                for i in range(self._n)]

----

In [11]:
t, n = 3, 5

In [12]:
d = ShamirDealer(t, n)

[TRACE] [ShamirDealer] Initializing superclass...
[TRACE] [GenericDealer] t = 3, n = 5
[TRACE] [GenericDealer] Finite Field of size 13257691184017152539829589394168647137633552917027648247059866272279644780342543886205702888517383139268204807182150416712174542674099220551207189888789539
[TRACE] [GenericDealer] key = 5776896569299686601712831586284508132104437214898259532992894663739617132723651790215998905818579851111643422084874920060963455859271769137589177434646873


In [13]:
participants = d.split_key()

[TRACE] [ShamirParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 1, y = 6020844153809243643995027639811967355982102718973442605223671012830872978166761666898495425301873794629949436594661386394441390988027348367606993907778404
[TRACE] [ShamirParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 2, y = 5109775815082134303305624710021990551648076826750053969741299667969274081309756191994413251699445905225169420063168947252420194730625480427897590043500485
[TRACE] [ShamirParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 3, y = 3043691553118358579644622796914577719102359538228093626545780629154820442152635365503752385011296182897303372490397602634899867087066165318460965841813116
[TRACE] [ShamirParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 4, y = 13080282551935069012841611294658375995978503770435209822696980168667156841037943073632215713754807766914556101058497769254054950731448623590504311191505836

In [14]:
restored_key = restore_secret_by_equation_system(d.field, 
                                                 random.sample(participants, t))
print(f'Restored key = {restored_key}')
print(f'Restored correctly: {restored_key == d.key}')

Restored key = 5776896569299686601712831586284508132104437214898259532992894663739617132723651790215998905818579851111643422084874920060963455859271769137589177434646873
Restored correctly: True


In [15]:
false_key = restore_secret_by_equation_system(d.field, 
                                              random.sample(participants, t - 1))
print(f'Restored key = {false_key}')
print(f'Restored correctly: {false_key == d.key}')

Restored key = 7509420454154686176170230061260662174421974309346117094562616204668899246173824817595866945447162600496272468646793278274212152938507939892180007940761048
Restored correctly: False


In [16]:
restored_key = restore_secret_by_lagrange_interpolation(random.sample(participants, t))
print(f'Restored key = {restored_key}')
print(f'Restored correctly: {restored_key == d.key}')

Restored key = 5776896569299686601712831586284508132104437214898259532992894663739617132723651790215998905818579851111643422084874920060963455859271769137589177434646873
Restored correctly: True


In [17]:
false_key = restore_secret_by_lagrange_interpolation(random.sample(participants, t - 1))
print(f'Restored key = {false_key}')
print(f'Restored correctly: {false_key == d.key}')

Restored key = 8086928415773019367656029552919380188527820007495402948419190051645326617323882493389156291990023516957815484167432731011961718631586663477043618109465773
Restored correctly: False


---
## Схема Фельдмана
![Feldman scheme](./images/Feldman_5.png)

In [18]:
class FeldmanParticipant(GenericParticipant):
    def __init__(self, x, y, t, g, check_values):
        """
        Тут в дополнение сохраняется генератор и проверочные значения.
        """
        
        self._g = g
        self._check_values = check_values
        
        trace('[FeldmanParticipant]', f'Initializing superclass...')
        super(FeldmanParticipant, self).__init__(x, y, t)
        
        
    def _verify(self):
        expected_value = self._g ^ self.y
        value_to_check = prod([cv ^ (self.x ^ i) for i, cv in enumerate(self._check_values)])
        
        if value_to_check != expected_value:
            raise ValueError(f'Wrong value given to the participant with x = {self.x} and y = {self.y}')
            
        trace('[FeldmanParticipant]', 'Value verified')

In [19]:
class FeldmanDealer(GenericDealer):
    def __init__(self, t, n):
        """
        В дополнение генерируются p, q, g.
        """
        
        self._p, self._q, self._g = FeldmanDealer._generate_parameters()
        
        trace('[FeldmanDealer]', f'Initializing superclass...')
        super(FeldmanDealer, self).__init__(t, n, GF(self._q))
        
        trace('[FeldmanDealer]', f'p = {self._p}')
        trace('[FeldmanDealer]', f'q = {self._q}')
        trace('[FeldmanDealer]', f'g = {self._g}')
        
        
    def split_key(self):
        """
        Разделение секрета между участниками.
        """
        
        self._prepare_coefficients()
        check_values = [self._g ^ c for c in self._coefs]
        
        return [FeldmanParticipant(self._field.gen() + i, self._values[i], self._t, self._g, check_values) 
                for i in range(self._n)]
    
    
    @staticmethod
    def _generate_parameters():
        def _generate_special_group_element(p, q):
            #
            # Сгенерирую число, взаимно простое с p
            #

            tmp = random.getrandbits(P_LENGTH)
            while gcd(tmp, p) != 1:
                tmp = random.getrandbits(P_LENGTH)

            #
            # По малой теореме Ферма такой g будет иметь порядок q
            #

            F = GF(p)
            return F(tmp) ^ ((p - 1) // q)
        
        Q_LENGTH = 160
        P_LENGTH = 1024
        
        q = getPrime(Q_LENGTH)
        p = random.getrandbits(P_LENGTH - Q_LENGTH) * q + 1
        
        while p not in Primes():
            p = random.getrandbits(P_LENGTH - Q_LENGTH) * q + 1
        
        return p, q, _generate_special_group_element(p, q)

----

In [20]:
d = FeldmanDealer(t, n)

[TRACE] [FeldmanDealer] Initializing superclass...
[TRACE] [GenericDealer] t = 3, n = 5
[TRACE] [GenericDealer] Finite Field of size 1215695783197808756755466617704054593913433544113
[TRACE] [GenericDealer] key = 352577935330903952972324104611874522430944059349
[TRACE] [FeldmanDealer] p = 110252752993766275067205834300575699464589431385714193480927203395481441524553507501160162146743984597625498494635741482352713443155786302198364880887057568856592866080186958339808243763312986841609539296940781359914742832914453987247340591023232130371220551343858059966145792382123885010348670395037971416433
[TRACE] [FeldmanDealer] q = 1215695783197808756755466617704054593913433544113
[TRACE] [FeldmanDealer] g = 455184582329812064231715634043706262843613534077760238941974063185455583574958114736786938531230181810866217388965137503217310500783765911797292677460504535724354428022122697069024856234195340009105802908582723014401459509882645459379243452399173461774756120501896556448691882061355330400357842

In [21]:
participants = d.split_key()

[TRACE] [FeldmanParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 1, y = 861249957178150645198879129148832322779506124622
[TRACE] [FeldmanParticipant] Value verified
[TRACE] [FeldmanParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 2, y = 213301499506616241155299793101166994488046401414
[TRACE] [FeldmanParticipant] Value verified
[TRACE] [FeldmanParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 3, y = 840124128711918254352519331876987725383431977951
[TRACE] [FeldmanParticipant] Value verified
[TRACE] [FeldmanParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 4, y = 310326278398439171279604510068185327638795766007
[TRACE] [FeldmanParticipant] Value verified
[TRACE] [FeldmanParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 5, y = 1055299514961796505447488563082868989081004853808
[TRACE] [FeldmanParticipant] Value verified


In [22]:
restored_key = restore_secret_by_equation_system(d.field, 
                                                 random.sample(participants, t))
print(f'Restored key = {restored_key}')
print(f'Restored correctly: {restored_key == d.key}')

Restored key = 352577935330903952972324104611874522430944059349
Restored correctly: True


In [23]:
false_key = restore_secret_by_equation_system(d.field, 
                                              random.sample(participants, t - 1))
print(f'Restored key = {false_key}')
print(f'Restored correctly: {false_key == d.key}')

Restored key = 977520681738436104874468151121614463610260047142
Restored correctly: False


In [24]:
restored_key = restore_secret_by_lagrange_interpolation(random.sample(participants, t))
print(f'Restored key = {restored_key}')
print(f'Restored correctly: {restored_key == d.key}')

Restored key = 352577935330903952972324104611874522430944059349
Restored correctly: True


In [25]:
false_key = restore_secret_by_lagrange_interpolation(random.sample(participants, t - 1))
print(f'Restored key = {false_key}')
print(f'Restored correctly: {false_key == d.key}')

Restored key = 263964979812362462244325718932727324520826425901
Restored correctly: False


----
## Схема Педерсена
![Pedersen scheme](./images/Pedersen_5.png)

In [26]:
class PedersenParticipant(GenericParticipant):
    def __init__(self, i, u, w, g, h, t, check_values):
        self._g, self._h = g, h
        self._w = w
        self._check_values = check_values
        
        trace('[PedersenParticipant]', f'Initializing superclass...')
        super(PedersenParticipant, self).__init__(i, u, t)
        
    
    def _verify(self):
        expected_value = (self._g ^ self.y) * (self._h ^ self._w)
        value_to_check = prod([cv ^ (self.x ^ i) for i, cv in enumerate(self._check_values)])
        
        if value_to_check != expected_value:
            raise ValueError(f'Wrong value given to the participant with i = {self.x}')
            
        trace('[PedersenParticipant]', 'Value verified')

In [27]:
class PedersenDealer(GenericDealer):
    def __init__(self, t, n):
        """
        """
        
        self._p, self._q, self._g, self._h = PedersenDealer._generate_parameters()
        self._gammas = []
        
        trace('[PedersenDealer]', f'Initializing superclass...')
        super(PedersenDealer, self).__init__(t, n, GF(self._q))
        
        trace('[PedersenDealer]', f'p = {self._p}')
        trace('[PedersenDealer]', f'q = {self._q}')
        trace('[PedersenDealer]', f'g = {self._g}')
        trace('[PedersenDealer]', f'h = {self._h}')
        
        
    def split_key(self):
        """
        """
        
        self._prepare_coefficients()
        self._gammas = [self._field.random_element() for _ in range(t)]
        
        check_values = [(self._g ^ d) * (self._h ^ g)
                        for d, g in zip(self._coefs, self._gammas)]
        
        return [PedersenParticipant(self._field.gen() + i, u, self._w(i + 1),
                                    self._g, self._h, self._t, check_values) 
                for i, u in enumerate(self._values)]
    
    
    def _w(self, z):
        return sum([g * (z ^ i) for i, g in enumerate(self._gammas)])
    
    
    @staticmethod
    def _generate_parameters():
        def _generate_special_group_element(p, q):
            #
            # Сгенерирую число, взаимно простое с p
            #

            tmp = random.getrandbits(P_LENGTH)
            while gcd(tmp, p) != 1:
                tmp = random.getrandbits(P_LENGTH)

            #
            # По малой теореме Ферма такой g будет иметь порядок q
            #

            F = GF(p)
            return F(tmp) ^ ((p - 1) // q)
        
        Q_LENGTH = 160
        P_LENGTH = 1024
        
        q = getPrime(Q_LENGTH)
        p = random.getrandbits(P_LENGTH - Q_LENGTH) * q + 1
        
        while p not in Primes():
            p = random.getrandbits(P_LENGTH - Q_LENGTH) * q + 1
            
        g = _generate_special_group_element(p, q)
        h = _generate_special_group_element(p, q)
        
        while g == h:
            h = _generate_special_group_element(p, q)
            
        return p, q, g, h

-----

In [28]:
d = PedersenDealer(t, n)

[TRACE] [PedersenDealer] Initializing superclass...
[TRACE] [GenericDealer] t = 3, n = 5
[TRACE] [GenericDealer] Finite Field of size 1365895551687335104465370138919923804890228099739
[TRACE] [GenericDealer] key = 180580734022488128228279271272451039328596880550
[TRACE] [PedersenDealer] p = 1541915044228927952972103517997595155469059565255117295059414727727736280510661266289587899607907880917874117284518182485031711397033835631806718546194699052101612426417965326804709499060360308457747634177716759172021142615833874676307014590943048234246394023150163498321751418907827204481391782916017946169
[TRACE] [PedersenDealer] q = 1365895551687335104465370138919923804890228099739
[TRACE] [PedersenDealer] g = 1434834480794992097742078535786103107409521638523631008929786508091716150034471974881086175546438264808694356878492145028236709181281957476746947358743265184286170175743640963585515030508001324839949495619530671718156043375293869462320907058613228901379683336749240493598548910365224275315990

In [29]:
participants = d.split_key()

[TRACE] [PedersenParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 1, y = 1090426921992062389180628958539187036373374034544
[TRACE] [PedersenParticipant] Value verified
[TRACE] [PedersenParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 2, y = 437830162263572077366228695002164423548941572544
[TRACE] [PedersenParticipant] Value verified
[TRACE] [PedersenParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 3, y = 954581558211687401715818758501230810635755694028
[TRACE] [PedersenParticipant] Value verified
[TRACE] [PedersenParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 4, y = 1274785558149073257764029010116462392743588299257
[TRACE] [PedersenParticipant] Value verified
[TRACE] [PedersenParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 5, y = 32546610388394541045489310927935364982211288492
[TRACE] [PedersenParticipant] Value verified


In [30]:
restored_key = restore_secret_by_equation_system(d.field, 
                                                 random.sample(participants, t))
print(f'Restored key = {restored_key}')
print(f'Restored correctly: {restored_key == d.key}')

Restored key = 180580734022488128228279271272451039328596880550
Restored correctly: True


In [31]:
false_key = restore_secret_by_equation_system(d.field, 
                                              random.sample(participants, t - 1))
print(f'Restored key = {false_key}')
print(f'Restored correctly: {false_key == d.key}')

Restored key = 1354896999892979351214413870441999954221164721057
Restored correctly: False


In [32]:
restored_key = restore_secret_by_lagrange_interpolation(random.sample(participants, t))
print(f'Restored key = {restored_key}')
print(f'Restored correctly: {restored_key == d.key}')

Restored key = 180580734022488128228279271272451039328596880550
Restored correctly: True


In [33]:
false_key = restore_secret_by_lagrange_interpolation(random.sample(participants, t - 1))
print(f'Restored key = {false_key}')
print(f'Restored correctly: {false_key == d.key}')

Restored key = 1359865110086864938036558142575459869202485978080
Restored correctly: False
